#### Digital Signal Processing Courseware: An Introduction (copyright © 2024)
## Authors: J. Christopher Edgar and Gregory A. Miller

Originally written in Mathematica by J. Christopher Edgar. Conversion to Jupyter Notebook by Song Liu.

The authors of this courseware are indebted to Prof. Bruce Carpenter (University of Illinois Urbana-Champaign). Bruce inspired the creation of this courseware, he consulted with the authors as this courseware was being developed, and he provided the original version of the code and text for several sections of this courseware (e.g. the section on complex numbers and the section on normal distributions). 

# <font color=red>DSP.06 The Fourier Transform</font>

# <font color=red>TUTORIAL</font>

### Setup

In [ ]:
# general imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image as img
from matplotlib import cm
from mpl_toolkits import mplot3d
from scipy.fft import fft, fftfreq
from sympy import Symbol, sin, series
from sympy import roots, solve_poly_system
import scipy.special
import matplotlib.patches as patches
import math
import cmath
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Figure size 
plt.rc("figure", figsize=(8, 6))

#function to create time course figure
#one waveform
def make_plot_1(x1,y1,type="b"): 
    plt.plot(x1, y1,type)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
#two overlaid waveforms with red and blue   
def make_plot_2(x1,y1,type1,x2,y2,type2): 
    plt.plot(x1, y1, type1)
    plt.plot(x2, y2, type2)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
#three overlaid waveforms with red, blue and green   
def make_plot_3(x1,y1,type1,x2,y2,type2,x3,y3,type3): 
    plt.plot(x1, y1, type1)
    plt.plot(x2, y2, type2)
    plt.plot(x3, y3, type3)
    plt.margins(x=0, y=0)
    plt.axhline(y=0, color='k')
    plt.tick_params(labelbottom = False, bottom = False)
    
def make_plot_3d(ax,x,y,z):    
    ax.contour3D(x, y, z, 50, cmap=cm.coolwarm)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
def make_plot_freq_1(x1,sample_rate, duration=1): 
    N = sample_rate * duration
    Nhalf = math.ceil(N/2)
    yf = fft(x1)
    xf = fftfreq(N, 1 / sample_rate)
    yf = yf[0:Nhalf]
    xf = xf[0:Nhalf]
    plt.plot(xf, np.abs(yf))
    
#two spectrums
def make_plot_freq_2(x1,x2,sample_rate, duration=1): 
    N = sample_rate * duration
    Nhalf = math.ceil(N/2)
    yf1 = fft(x1)
    yf2 = fft(x2)
    xf = fftfreq(N, 1 / sample_rate)

    yf1 = yf1[0:Nhalf]
    yf2 = yf2[0:Nhalf]
    xf = xf[0:Nhalf]

    plt.plot(xf, np.abs(yf1))
    plt.plot(xf, np.abs(yf2), color = 'r')
    
def make_imshow(x):
    plt.imshow(x,cmap='Greys_r')
    plt.tick_params(labelbottom = False, bottom = False)
    plt.tick_params(labelleft = False, left = False)
    
def make_imshow_color(x):
    plt.imshow(x)
    plt.tick_params(labelbottom = False, bottom = False)
    plt.tick_params(labelleft = False, left = False)
    
def round_complex(x):
    return complex(np.round(x.real,4),np.round(x.imag,4))

In the Basics we saw how the Discrete Fourier Transform works. We noted that it determines a set of frequencies that completely describe the timeseries - a model of the timeseries. Although the Fourier
transform always produces an answer, there are a number of instances where the Fourier seems to
'fail'. Actually, the Fourier transform doesn't fail - how can it, it's just a mathematical procedure,
it does what we tell it to. The problem is that people can apply the Fourier transform or interpret
the Fourier results incorrectly.

## <font color=red>DSP.06.T1) The Discrete Fourier Transform (DFT): Some Issues</font>

### <font color=red>DSP.06.T1.a) “Leakage” and its relationship to frequency resolution (Δf)</font>

Let's look at a the Fourier transform of a 15 Hz timeseries.

In [ ]:
time = np.arange(0,1,0.01)
timeseries = 5 * np.sin(2*np.pi * 15 * time)
make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)
plt.show()

In [ ]:
make_plot_freq_1(timeseries,100)

# Adding text to the figure
plt.text(51,0,'Hz',fontsize=15)
plt.show()

The peak is at 15 Hz - what we expected.

Now look at a the Fourier transform of a 15.5 Hz timeseries

In [ ]:
time = np.arange(0,1,0.01)
timeseries =  5 * np.sin(2*np.pi * 15.5 * time) 
make_plot_1(time,timeseries)
plt.text(1,0,'1 sec',fontsize=15)
plt.show()

In [ ]:
make_plot_freq_1(timeseries,100)

# Adding text to the figure
plt.text(52,0,'Hz',fontsize=15)
plt.show()

Overlay the two spectra. You might expect them to look the same, except for the second one being shifted 0.5 Hz to the right.

In [ ]:
timeseries1 = 5 * np.sin(2*np.pi * 15 * time) 
timeseries2 = 5 * np.sin(2*np.pi * 15.5 * time) 
make_plot_freq_2(timeseries1,timeseries2,100)

sample_rate = 100
duration = 1

plt.text(52,0,'Hz')
plt.show()

Look closely at the red (15.5 Hz) frequency spectrum plot. The peak is at ~15 Hz, but: the peak is
smaller, the peak is broader, and there also appears to be some activity at frequencies to the left and right of the
peak. Why is that?

Look again at the Discrete Fourier Transform formula

$F[k Δf] =  \frac{\sum_{n=0}^{N-1}e^{i(2 \pi k Δf) (nΔt)}}{\sqrt{N}}$ for k = 0,1,2,..., N-1

In Lesson 6 Basics, we saw that ω corresponds to Δf, where Δf = 1/T = frequency resolution. This tells us
that Δf depends only on the epoch length (i.e., time). The longer the epoch being analyzed, the greater our frequency
resolution. The shorter the epoch, the poorer our frequency resolution. This is an important point, which we noted in Basics:
increasing the sample rate improves temporal resolution, but it doesn’t improve frequency resolution
in the way you might think. It improves frequency resolution in the sense that it raises the Nyquist
frequency, so we can identify higher and higher frequencies. But it doesn’t improve the resolution in
terms of how small a fraction of frequency it can resolve.

Here are some examples:
    
A 0.5 second epoch provides a frequency resolution (Δf ) of 1/T = 1/0.5 = 2. Thus, we obtain a measure every
2 Hz.

A 3 second epoch provides a frequency resolution (Δf ) of 1/3 = 0.333. Thus, we obtain a measure every 1/3
Hz.

A 10 second epoch provides a frequency resolution (Δf ) of 1/10 = 0.1. Thus, we obtain a measure every 1/10
Hz.

A 50 second epoch provides a frequency resolution (Δf ) of 1/50 = 0.1. Thus, we obtain a measure every 1/50
Hz.

Look at the 2 frequency spectra again.

In [ ]:
timeseries1 = 5 * np.sin(2*np.pi * 15 * time) 
timeseries2 = 5 * np.sin(2*np.pi * 15.5 * time) 
make_plot_freq_2(timeseries1,timeseries2,100)

sample_rate = 100
duration = 1

plt.text(52,0,'Hz')
plt.show()

The activity in frequencies surrounding 15.5 Hz is called leakage. This appears when the
vector of data samples does not stop at exactly the same phase in the sine wave as it started. If an
infinite number of discrete samples were taken, leakage would not be a problem. However, in the real
world (where we collect a finite number of data points) there will sometimes be leakage.

We’ll drill into this problem of leakage in a moment. First, let’s frame the problem more intuitively. We
can think of the Fourier approach as in effect building a model of a raw data vector out of a series of
specific frequencies. Like any model, that model can do a better or worse job of capturing the major or
most important features of what it’s modeling. In the examples we’ve just looked at, the Fourier
approach did a great job modeling the 15 Hz timeseries but apparently not nearly as good a job modeling
the 15.5 Hz time series.

Before you go on, be careful about how you understand the problem we've just framed: the models for the 15 Hz case and the 15.5 Hz case are equally successful, if your criterion for success is whether an Inverse Fourier Transform of the result of the Discrete Fourier Transform would precisely reproduce the original timeseries. It would! But the spectrum in the 15.5 Hz case doesn't look right. It's not obvious why the 15 Hz spectrum is nice and pointy, but the 15.5 Hz spectrum kind of blurs across the spectrum. We're about to figure that out. But don't confuse (1) whether the DFT model captures all the information in the 15.5 Hz timeseries, in that the timeseries can be recreated from it (it can), with (2) whether the plot of the DFT result looks as it should (it doesn't).

OK, back to work. It doesn’t seem obvious why the DFT would seem to behave so differently with two such
similar timeseries. What’s up with that?

### <font color=red>DSP.06.T1.b) Reducing leakage by increasing the epoch length</font>

As we saw above, we can decrease Δf by increasing the epoch length. Try applying the Fourier transform
to a longer epoch. We can do this by increasing the variable 'points'. (Note that we're not trying to solve the problem by sampling faster. We don't have an aliasing problem. We're trying to solve the problem by changing the length of the timeseries we're going to analyze.)

In [ ]:
time = np.arange(0,3,0.01)
timeseries = 5 * np.sin(2*np.pi * 15.5 * time)  
make_plot_1(time,timeseries)
plt.text(3,0,'3 secconds',fontsize=15)
plt.show()

The epoch length is 3 seconds. As such, the frequency resolution is Δf = 1/T = 1/3 secs = 0.33 Hz. Thus, we
obtain information in 0.3 Hz steps. What effect does this have on leakage?

Calculate the Fourier transform and take a look at the results. 

In [ ]:
SAMPLE_RATE = 100
DURATION  = 3

N = SAMPLE_RATE * DURATION
Nhalf = round(N/2)

yf3 = fft(timeseries)
yf3 = yf3[0:Nhalf]
xf3 = fftfreq(N, 1 / SAMPLE_RATE)
xf3 = xf3[0:Nhalf]

plt.plot(xf3, np.abs(yf3))

# Adding text to the figure
plt.text(52,0,'Hz', fontsize=15)
plt.show()

A little better, but there is still leakage. Try again using an even longer 5 second epoch.

In [ ]:
time = np.arange(0,5,0.01)
timeseries = 5 * np.sin(2*np.pi * 15.5 * time)  
make_plot_1(time,timeseries)
plt.text(5,0,'5 secconds',fontsize=15)
plt.show()

The epoch length is 5 seconds. So, the frequency resolution Δf = 1/T = 1/5 secs = 0.2 Hz. Thus, we obtain information
in 0.2 Hz steps. What effect does this have on leakage?

Calculate the Fourier transform and look at the results.

In [ ]:
SAMPLE_RATE = 100
DURATION = 5

N = SAMPLE_RATE * DURATION
Nhalf = round(N/2)

yf5 = fft(timeseries)
yf5 = yf5[0:Nhalf]
xf5 = fftfreq(N, 1 / SAMPLE_RATE)
xf5 = xf5[0:Nhalf]

plt.plot(xf5, np.abs(yf5))

# Adding text to the figure
plt.text(52,0,'Hz')
plt.show()

Still a little leakage, but this looks pretty good. Theoretically, we can increase our epoch length until we
are satisfied. Practically, however, we have only so much data, so there comes a point where we have
to stop. Also, the longer our epoch, the more likely the signal will change during the interval we’re
analyzing (i.e., the less likely the signal will be "stationary").

That’s a problem because the Fourier Transform assumes that our signals are constant (stationary) during the analysis epoch. If our signal is stationary, then the DFT will still restore the original signal with high fidelity. But a non-stationary signal means that a model made up of stationary sinusoids will inherently be inaccurate as a description of the original signal.

Show the above three spectrums on the same plot.

In [ ]:
time1 = np.arange(0,1,0.01)
x1 = 5 * np.sin(2*np.pi * 15.5 * time1) 
N = 100*1
Nhalf = math.ceil(N/2)
yf1 = fft(x1)
xf1 = fftfreq(N, 0.01)
yf1 = yf1[0:Nhalf]
xf1 = xf1[0:Nhalf]

plt.plot(xf1, np.abs(yf1),'g',xf3, np.abs(yf3),'b',xf5, np.abs(yf5),'k') 
plt.text(52,0,'Hz')
plt.show()

Notice that, as the epoch length increases (black being the longest epoch we tried), the 15 Hz peak is
increasingly sharp. The above suggests that, to the extent possible, a longer epoch length is preferred.

Finally, consider the following: with a 15.5 Hz timeseries, to accurately assess 15.5 Hz activity we need a
frequency resolution of 0.5 Hz. Set the duration to 2 seconds to obtain 0.5 Hz frequency resolution.

In [ ]:
time = np.arange(0,2,0.01)
timeseries = 5 * np.sin(2*np.pi * 15.5 * time)  
make_plot_1(time,timeseries)
plt.text(2,0,'2 secconds',fontsize=15)
plt.show()

The epoch length is 2 seconds. The frequency resolution Δf = 1/T = 1/2 secs = 0.5 Hz. Thus, we obtain information
in 0.5 Hz steps. What effect does this have on leakage?

Calculate the Fourier transform and look at the results. 

In [ ]:
SAMPLE_RATE = 100
DURATION = 2

N = SAMPLE_RATE * DURATION
Nhalf = round(N/2)

yf2 = fft(timeseries)
yf2 = yf2[0:Nhalf]
xf2 = fftfreq(N, 1 / SAMPLE_RATE)
xf2 = xf2[0:Nhalf]

plt.plot(xf2, np.abs(yf2))

# Adding text to the figure
plt.text(52,0,'Hz', fontsize=15)
plt.show()




Overlay all the above power spectrum.

In [ ]:
plt.plot(xf1, np.abs(yf1),'g',xf3, np.abs(yf3),'b',xf5, np.abs(yf5),'k',xf2, np.abs(yf2),'r') 
plt.text(52,0,'Hz')
plt.show()   

The 2 second epoch (shown in red) looks the best. This is because with 2 seconds 15.5 Hz activity is assessed at exactly
15.5 Hz.

We can summarize the above. The Fourier approach to analyzing a timeseries of length T (in seconds)
is built around a sine wave of frequency 1/T (in cycles per second) and its harmonics. (0/T is also used - it gets the mean of the whole epoch = also called the DC level - though for many real-world analysis situations that has 0 magnitude, because the mean is zero.) In the example
above, our first frequency resolution was 1 Hz, because the epoch was 1 second. As a result we obtain
measures at 1 Hz, 2 Hz, 3 Hz, etc...

Fourier modeling of the time series will work properly only when the slowest frequency in the data,
other than overall mean level (0 Hz), is exactly 1/T. In other words, Fourier assumes that there is a frequency
contributing to the activity in epoch T which has a cycle length exactly equal to T. Furthermore, all
other (faster) frequencies in the data are assumed to be limited to the harmonics 2/T, 3/T, etc. In the
output of the Discrete Fourier Transform, the longer the epoch T is, the finer the frequency resolution of
the Fourier Transform.

We'll say it once more: When the slowest frequency in the timeseries isn't 1/T, the spectrum resulting from the DFT doesn't misrepresening the time series in the sense that the IFT applied to that spectrum will still reproduce the original timeseries. But the model that the DFT spectrum suggests won't be accurate. Frequencies will leak into spectrum bins in a way that could be misinterpreted. So, remember this issue of leakage.

### <font color=red>DSP.06.T1.c) Reducing leakage: does increasing the sampling rate help?</font>

Can we also decrease the amount of leakage if we collect data points more often?

Answer: If you think through what we learned about increasing fs, you can figure this out without having to do
any calculations. But if it's not clear, try an experiment. Take the 1 second signal we started with but
sample at 10 times the original sample rate.

In [ ]:
time = np.arange(0,1,0.001)
timeseries = 5 * np.sin(2*np.pi * 15.5 * time)  
make_plot_1(time,timeseries)
plt.text(1,0,'1 seccond',fontsize=15)
plt.show()

Plot the frequency spectrum for this 1 second of data.

In [ ]:
SAMPLE_RATE = 1000
DURATION = 1

N = SAMPLE_RATE * DURATION
Nhalf = round(N/2)

yf2 = fft(timeseries)
yf2 = yf2[0:Nhalf]
xf2 = fftfreq(N, 1 / SAMPLE_RATE)
xf2 = xf2[0:Nhalf]

plt.plot(xf2, np.abs(yf2))

# Adding text to the figure
plt.text(502,0,'Hz', fontsize=15)
plt.show()

Because we sampled at 1000 Hz, the x axis extends to 500 Hz (the Nyquist frequency).

Re-plot the frequency spectrum going out to only 50 Hz.

In [ ]:
SAMPLE_RATE = 1000
DURATION = 1

N = SAMPLE_RATE * DURATION
Nhalf = round(N/2)

yf2 = fft(timeseries)
yf2 = yf2[0:round(Nhalf/10)]
xf2 = fftfreq(N, 1 / SAMPLE_RATE)
xf2 = xf2[0:round(Nhalf/10)]

plt.plot(xf2, np.abs(yf2))

# Adding text to the figure
plt.text(52,0,'Hz', fontsize=15)
plt.show()

Leakage is still present. In fact, increasing fs didn't help the leakage at all. This shouldn't be a surprise; remember - increasing fs only increases the number of frequencies examined. If you want to reduce leakage, you need to increase the epoch length, thus decrease the width of each frequency bin in the spectrum, thus improve the frequency resolution.

### <font color=red>DSP.06.T1.d) Summing Up</font>

Fourier analysis is happiest when the Fourier transform is applied to a timeseries of infinite length,
because this leakage into inappropriate frequency bins will not occur. How come? Well, as T approaches infinity, 1/T approaches 0.0. As a result, the width of each frequency
bin approaches zero. As a result, the frequency resolution becomes extremely high, so that virtually any
activity is close to a harmonic of 1/T. Very long analysis epochs are thus much less vulnerable to leakage of
non-harmonic activity.

On the other hand, longer analysis epochs are more vulnerable to violation of the stationarity assumption (no
changes in the constituent frequencies over time - their magnitude or phase). The Fourier transform from the time to the frequency
domain produces a set of magnitude and phase values, one magnitude and one phase value for each
harmonic. Because the entire time series will be described by a (static) set of frequencies of specified
amplitude and phase, this approach can't represent correctly, in a set of such static sine waves, any change in the amplitude or phase
of a given frequency during the T epoch. It WILL represent the original data fully, because the IFT can recover it fully. But the model will be misleading (if you forget to consider the issues discussed in this Lesson 6 Tutorial).

If you want a spectrum that's straightforward to interpret, the data you're analyzing must be stationary during the epoch
analyzed. (i\If you want to review the difference between stationary and non-stationary timeseries, go back
to the Lesson 5 Tutorial).

You can see that Fourier analysis isn’t a magic wand that can handle all cases with equal ease. You
can apply it to any timeseries. You can do the forward transform, and then do the inverse transform, and you’ll get back your original data. In that sense, Fourier analysis doesn’t distort
the original data. Very impressive. But the model, in the frequency domain, that Fourier analysis produces may be
misleading if you’re not careful about epoch length (thus frequency resolution) and stationarity. In fact,
you can appreciate that epoch length and stationarity often trade off - the shorter the epoch, the more
likely the data are to be stationary, but the less frequency resolution you’ll have. Depending on your data and your analysis goals, that may or may not be a problematic trade-off.

## <font color=red>DSP.06.T2) One More Issue</font>

### <font color=red>DSP.06.T2.a) Our Old Friend Aliasing Returns</font>

Above we saw an example where, because the epoch length wasn't long enough, the Fourier transform
produced misleading results = 'incorrect' results, if you were asking Fourier to tell you what frequencies were actually in your data (not merely asking it to model the data, which it can do just fine). We said above that there's more than one way to go wrong. Here's
another.

In Chapter 2 we discussed aliasing and what happens when we don't sample at a high enough sampling
rate. Let's review this topic one more time. Look at this timeseries.

In [ ]:
time = np.arange(0,1,0.01)
timeseries = np.sin(2*np.pi * 10 * time) + np.sin(2*np.pi * 97 * time) 
make_plot_1(time,timeseries)
plt.text(1,0,'1 seccond',fontsize=15)
plt.show()

Can you detect the problem? (Hint: What frequencies did we build into that timeseries, and what sampling rate did we use?) If not, keep going.

Calculate the Fourier transform and plot the power spectrum

In [ ]:
SAMPLE_RATE = 100
DURATION = 1

N = SAMPLE_RATE * DURATION
Nhalf = round(N/2)

yf2 = fft(timeseries)
yf2 = yf2[0:Nhalf]
xf2 = fftfreq(N, 1 / SAMPLE_RATE)
xf2 = xf2[0:Nhalf]

plt.plot(xf2, np.abs(yf2))

# Adding text to the figure
plt.text(52,0,'Hz', fontsize=15)
plt.show()

There are 2 distinct peaks - one at 3 Hz, one at 10 Hz. But we created this timeseries, so we know there
is activity only at 10 and 97 Hz. Why does the spectrum show activity at 3 Hz? Why doesn't the Fourier
transform show the 97 Hz peak? (Check out the code cell above that generated the time series, and find the multipications by 10 and by 97 to confirm that those are the 2 frequencies we put into this timeseries.)

Answer: In Lesson 2, we noted that, when sampling at some sampling frequency fs, we accurately represent
information for frequencies only up to fs/2, known as the Nyquist frequency. In the example above, we
sampled at 100 Hz. This means that we can obtain reliable frequency information only for frequencies
less than fs/2 = 100/2 = 50 Hz. Any activity in the timeseries that’s faster than 50 Hz will be aliased. Typically,
this will mean that it shows up as activity at the frequency [sample rate minus signal], which in this
case is 100 - 97 = 3.

If we want to represent the 97 Hz activity without aliasing, we'll need to sample at a higher rate. In
other words, we have to have a Nyquist frequency higher than 97 Hz.

You might wonder: If we sample less than twice the fastest activity in the raw (probably continuous) phenomenon, then doesn't that mean that the DFT-then-IFT sequence will fail to recreate the original data? Not so split hairs, but: no, it doesn't mean that. The problem isn't the DFT failing to retain all the information in the digitized timeseries. The problem is the digitized timeseries failing to represent all the information in the original phenomenon. Whether or not your digitized timeseries aliased the original phenomenon, DFT-then-IFT will recreate the digitized timeseries (warts and all). In the present case, the timeseries resulting from digitzing at 100 Hz can't have stored the (unaliased) 97 Hz component of the signal. It's the sampling that missed the 97 Hz activity, not the DFT. Don't blame Fourier for someone (you???) failing to digitize fast enough. 

Give it a try. To accurately measure the 97 Hz activity, we'll have to collect at least 194 points/sec. To be
safe, let's sample at 200 Hz.

In [ ]:
time = np.arange(0,1,1/200)
timeseries = np.sin(2*np.pi * 10 * time) + np.sin(2*np.pi * 97 * time)
make_plot_1(time,timeseries)
plt.text(1,0,'1 seccond',fontsize=15)
plt.show()

Calculate the Fourier transform and plot the power spectrum

In [ ]:
make_plot_freq_1(timeseries,200)

# Adding text to the figure
plt.text(102,0,'Hz',fontsize=15)
plt.show()

That's better. Now there are two distinct peaks, and they are in the right place - one at 10 Hz and one at
97 Hz.